# Linear Regression

In this labs, we are going to apply linear regression to the problem of predicting developer satisfaction based upon information about their careers, from a StackOverflow survey.  The data from this question is based on the [2019 StackOverflow Survey](https://insights.stackoverflow.com/survey/2019); accordingly, the subset bundled with this assignment is also released under the Open Database License (ODbL) v1.0.  For this problem, you should not use Scikit-Learn, but instead, implement all the least squares solutions manually.


### Q1 Data Parsing

The data from this survey has the values as described below.  Your eventual goal will be to predict (a numerical equivalent of) the `CarreerSat` column, indicating how satisfied the subject is with their career), based upon the other values.  Note that because the career satisfaction values are ordinal, we likely should not be predicting them with linear regression, but as an illustrative example, this is still a reasonable task.

The data set contains the following columns.

| Column | Sample | Does/is the respondent... | Type/Values |
| --- |:--- |:--- |:--- |
| **CareerSat** | 'vs' | satisfied with their career? | (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) -- corresponding to ({very, slightly}, {satisfied, dissatisfied}), neutral, and not applicable |
| MgrWant | 'n' | ...want to be a manager? | boolean |
| Age    | '22'   | age | integer     |
| CodeRevHrs | '2' | hours a week spent reviewing code | integer |
| ConvertedComp | '61000' | yearly compensation in 2019 USD | integer |
| Country | 'United States' | lives in country | string _(ignore in regression)_ |
| Dependents | 'n' | ...have children or other dependents. | boolean |
| DevEnvironVSC | 'y' | ...use Visual Studio Code | boolean |
| DevTypeFullStack | 'n' | ...identify as a full-stack developer | boolean |
| EdLevel | 'bachelors' | maximum education level | (`other`, `bachelors`, `masters`, `doctoral`) |
| EduOtherMOOC | 'y' | ...ever taken a Massively Open Online Course | boolean |
| EduOtherSelf | 'y' | ...ever taught themselves a new platform | boolean _(ignore in regression, this is  duplicate field)_ |
| Extraversion | 'y' | ...prefer in-person meetings to online meetings | boolean |
| GenderIsMan | 'y' | ...male | boolean |
| Hobbyist | 'n' | ...write code as a hobby? | boolean |
| MgrIdiot | 'very' | ...think their manager knows what they are doing? | (`NA`, `not`, `some`, `very`), in order of increasing confidence |
| OpSys | 'win' | which OS do they use? | (`win`, `mac`, `tux`, `NA`), for (Windows, Mac OSX, Linux-like, NA) |
| OpenSourcer | 'Never' | ...contribute to open-source projects? | (`never`, `year`, `month-year`, `month`), in increasing order of frequency |
| OrgSize | '100-499' | number of employees in organization? | (`NA`, `1`, `2-9`, `10-19`, `20-99`, `100-499`, `500-999`, `1,000-4,999`, `5,000-9,999`, `10,000+`) |
| Respondent | '4' | respondent ID from original data | integer _(ignore in regression)_ |
| Student | 'n' | ...currently a student? | boolean |
| UndergradMajorIsComputerScience | 'y' | ...majored in CS? | boolean |
| UnitTestsProcess | 'n' | ...use unit tests in their job? | boolean |
| WorkWeekHrs | '40' | hours a week worked | integer |
| YearsCode | 3 | years since first programming | integer |
| YearsCodePro | 0 | years programming professionally | integer |

When you load the data for linear regression, you will want to convert each type to a floating point value according to the columns above.  Specifically, for each column listed above, convert the value to a numeric quantity using the rules below.  Note that there are some fairly unnatural assumptions here (like converting NAs to 0.0), but these are largely to illustrate the methodology without adding too much additional processing work.

 - _boolean_ : `y`/`NA`/`n` assigned to `+1.0`/`0.0`/`0.0`
 - _integer_ : convert to `float`, preserving value. `NA` equals `0.0`. 
 - _string_ : not included in regression; we'll use it later
 - CareerSat: Map (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) to (-2.0, -1.0, 0.0, 0.0, 1.0, 2.0)
 - EdLevel: Map (`other`, `bachelors`, `masters`, `doctoral`) to (0.0, 1.0, 1.5, 2.0)
 - MgrIdiot: Map (`NA`, `not`, `some`, `very`) to (-1.0, -1.0, 0.0, 1.0)
 - OpSys: Map (`win`, `mac`, `NA`, `tux`, `BSD`) to (-1.0, 0.0, 0.0, 1.0, 1.0)
 - OpenSourcer : Map (`never`, `year`, `month-year`, `month`) to (0.0, 0.5, 1.0, 2.0)
 - OrgSize: Map each range "$a$-$b$" to the value $ln(a)$. Treat `NA` as `ln(1.0) = 0`. We are converting an exponentially distributed range to a linearly distributed one.

Remove the columns listed above as being ignored. 

Some hints:
  1. Load the csv file with `pd.read_csv(filname, dtype=str, keep_default_na=False)` to ensure that you load all columns as text (so you can do your own preprocessing), and ignore pandas's default conversion to NaN values.
  2. Use the `.apply()` function in pandas to convert

In [ ]:
import csv
import gzip
import math
import hashlib
import numpy as np
import pandas as pd

In [ ]:

def parse_stackoverflow_data(filename="eggs.csv.gz"):
    """
    Load data from the given file, and convert for use in linear regression.

    Args:
        filename (str): Path to the CSV file (gzip compressed).

    Returns:
        pandas.DataFrame: DataFrame with the data converted to floating-point values appropriately.
    """
    # Load the data as strings, prevent automatic NA conversion
    data = pd.read_csv(filename, dtype=str, keep_default_na=False)
    
    # Mappings for categorical and ordinal data
    career_sat_map = {'vd': -2.0, 'sd': -1.0, 'ne': 0.0, 'NA': 0.0, 'ss': 1.0, 'vs': 2.0}
    ed_level_map = {'other': 0.0, 'bachelors': 1.0, 'masters': 1.5, 'doctoral': 2.0}
    mgr_idiot_map = {'NA': -1.0, 'not': -1.0, 'some': 0.0, 'very': 1.0}
    opsys_map = {'win': -1.0, 'mac': 0.0, 'NA': 0.0, 'tux': 1.0, 'BSD': 1.0}
    opensourcer_map = {'never': 0.0, 'year': 0.5, 'month-year': 1.0, 'month': 2.0}

    # Convert "OrgSize" to log-transformed values
    def orgsize_to_log(value):
        if value == 'NA':
            return 0.0
        try:
            lower_bound = int(value.split('-')[0].replace(',', ''))
            return math.log(lower_bound)
        except ValueError:
            if value == '10,000+':
                return math.log(10000)
            return 0.0

    # Convert boolean columns ('y', 'n', 'NA') to float (1.0, 0.0, 0.0)
    def boolean_to_float(value):
        return 1.0 if value == 'y' else 0.0

    # Convert columns
    data['CareerSat'] = data['CareerSat'].map(career_sat_map).astype(float)
    data['MgrWant'] = data['MgrWant'].apply(boolean_to_float)
    data['Age'] = data['Age'].replace('NA', '0').astype(float)
    data['CodeRevHrs'] = data['CodeRevHrs'].replace('NA', '0').astype(float)
    data['ConvertedComp'] = data['ConvertedComp'].replace('NA', '0').astype(float)
    data['Dependents'] = data['Dependents'].apply(boolean_to_float)
    data['DevEnvironVSC'] = data['DevEnvironVSC'].apply(boolean_to_float)
    data['DevTypeFullStack'] = data['DevTypeFullStack'].apply(boolean_to_float)
    data['EdLevel'] = data['EdLevel'].map(ed_level_map).astype(float)
    data['EduOtherMOOC'] = data['EduOtherMOOC'].apply(boolean_to_float)
    data['Extraversion'] = data['Extraversion'].apply(boolean_to_float)
    data['GenderIsMan'] = data['GenderIsMan'].apply(boolean_to_float)
    data['Hobbyist'] = data['Hobbyist'].apply(boolean_to_float)
    data['MgrIdiot'] = data['MgrIdiot'].map(mgr_idiot_map).astype(float)
    data['OpSys'] = data['OpSys'].map(opsys_map).astype(float)
    data['OpenSourcer'] = data['OpenSourcer'].map(opensourcer_map).astype(float)
    data['OrgSize'] = data['OrgSize'].apply(orgsize_to_log)
    data['Student'] = data['Student'].apply(boolean_to_float)
    data['UndergradMajorIsComputerScience'] = data['UndergradMajorIsComputerScience'].apply(boolean_to_float)
    data['UnitTestsProcess'] = data['UnitTestsProcess'].apply(boolean_to_float)
    data['WorkWeekHrs'] = data['WorkWeekHrs'].replace('NA', '0').astype(float)
    data['YearsCode'] = data['YearsCode'].replace('NA', '0').astype(float)
    data['YearsCodePro'] = data['YearsCodePro'].replace('NA', '0').astype(float)

    # Drop ignored columns
    ignored_columns = ['Country', 'EduOtherSelf', 'Respondent']
    data = data.drop(columns=ignored_columns)

    return data

# Test the function (sample file path provided for clarity)
# df = parse_stackoverflow_data("eggs.csv.gz")
# df.head()


### Q2 Splitting Data

Now we prepare the converted data for regression. In this step, we:

 1. Extract the data as a numpy array
 2. Split the data into train and validation sets.  You can use the first 20% of the data (rounded down) as the validation set and keep the remaining as the training set. (Note that it is common practice to randomize the dataset; this has already been done. Don't shuffle the dataset for this assignment.)
 3. Split each set into the predicted column (the first column in the data frame) and the feature columns (the remaining columns), plus a final column corresponding to a constant 1.0 value.  Not that you should keep the ordering of the feature columns the same as they appear in the data.


In [ ]:
def split_data(df):
    """
    Split the data into training and validation sets, and convert them to np.ndarray.

    Args:
        df: pandas.DataFrame -- the parsed data, as returned by parse_stackoverflow_data()

    Returns:
        X_train, y_train, X_val, y_val:
          - X_train: np.ndarray -- the second 80% of the data features
          - y_train: np.ndarray -- the second 80% of the target values
          - X_val: np.ndarray -- the first 20% (rounded down) of the data features
          - y_val: np.ndarray -- the first 20% of the target values
    """
    # Convert DataFrame to numpy array
    data = df.to_numpy()

    # Determine split index
    split_index = len(data) // 5  # 20% of the data

    # Split into validation and training sets
    val_data = data[:split_index]
    train_data = data[split_index:]

    # Separate features (columns 1 onward) and target (column 0)
    X_val = val_data[:, 1:]
    y_val = val_data[:, 0]
    X_train = train_data[:, 1:]
    y_train = train_data[:, 0]

    # Add a constant column of 1.0 to features for the bias term
    X_val = np.hstack((X_val, np.ones((X_val.shape[0], 1))))
    X_train = np.hstack((X_train, np.ones((X_train.shape[0], 1))))

    return X_train, y_train, X_val, y_val

# Testing with example data would require an appropriate DataFrame.


### Q3 Linear Regression

Now we are going to build a simple scikit-learn-like class for least squares linear regression.  Recall from lecture that the linear regression approach models the data as:
$$ y^{(i)} \approx \theta^T x^{(i)} $$
and the optimal $\theta$ is given by:
$$ \theta^* = (X^T X)^{-1}X^T y $$
using the notation described in the slides and course notes.  Recall, as mentioned in class, that you should use the `np.linalg.solve()` function rather than the `np.linalg.inv()` function to compute this solution.

Implement the class below, plus the `squared_error` function.

In [ ]:
def squared_error(y_pred, y):
    """
    Compute the average squared error between predictions and ground truth.

    Args:
        y_pred: np.ndarray[num_examples] -- the predictions
        y: np.ndarray[num_examples] -- the ground truth values

    Returns:
        float: Average squared error between y_pred and y.
    """
    return np.mean((y_pred - y) ** 2)


class LinearRegression:
    """
    Perform linear regression and predict the output on unseen examples.

    Attributes:
        theta (np.ndarray): Vector containing parameters for the features.
    """

    def __init__(self, X, y):
        """
        Train the linear regression model by computing the estimate of the parameters.

        Args:
            X: np.ndarray[num_examples, num_columns] -- Matrix of training data.
            y: np.ndarray[num_examples] -- Vector of output variables.
        """
        # Compute theta using the closed-form solution (normal equation)
        self.theta = np.linalg.solve(X.T @ X, X.T @ y)

    def predict(self, X):
        """
        Use the learned model to predict the output of X.

        Args:
            X: np.ndarray[num_examples, num_columns] -- Matrix of features for prediction.

        Returns:
            np.ndarray[num_examples]: Vector of predicted outputs.
        """
        return X @ self.theta


# Example usage (assuming X_train, y_train, X_val, y_val are prepared):
# model = LinearRegression(X_train, y_train)
# predictions = model.predict(X_val)
# error = squared_error(predictions, y_val)
# print("Validation Error:", error)


## Q4 Evaluation versus baselines

As a final consideration, If you implement this properly, you will see that we get a squared error of around 1.3 on the validation set.  Is this "good"?  This is one of the more subtle points of many data science problems, but we can start to get some sense of this by looking at what the predictions would look like if we _just_ predicted the mean target output on the training set (this is essentially the "simplest" prediction we could make if we didn't look at the features at all).

Implement the following function to evaluate our linear regression.  Think about what this signifies about the quality of the solution.

In [ ]:
def evaluate_linear_regression(X_train, y_train, X_val, y_val):
    """
    Evaluate the squared error of linear regression versus the simple mean-prediction baseline.

    Args:
        X_train: np.ndarray -- Training feature matrix (num_examples, num_columns)
        y_train: np.ndarray -- Training target values (num_examples,)
        X_val: np.ndarray -- Validation feature matrix (num_examples, num_columns)
        y_val: np.ndarray -- Validation target values (num_examples,)

    Returns:
        Tuple[validation_mse, baseline_mse]:
            validation_mse: float -- Squared error of predictions on validation set, when training on training set.
            baseline_mse: float -- Squared error of predicting the mean of the training set on the validation set.
    """
    # Train the LinearRegression model
    model = LinearRegression(X_train, y_train)
    predictions = model.predict(X_val)
    
    # Compute validation mean squared error for the model
    validation_mse = squared_error(predictions, y_val)

    # Compute baseline mean squared error (predicting the mean of y_train)
    baseline_prediction = np.mean(y_train)
    baseline_mse = squared_error(np.full_like(y_val, baseline_prediction), y_val)

    return validation_mse, baseline_mse


# Example usage (requires prepared data):
# validation_mse, baseline_mse = evaluate_linear_regression(X_train, y_train, X_val, y_val)
# print("Validation MSE:", validation_mse)
# print("Baseline MSE:", baseline_mse)
